In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False


In [ ]:
if IN_COLAB:
    !git clone https://github.com/Gaurang8200/Projekt_AIM_1.git

fatal: destination path 'Projekt_AIM_1' already exists and is not an empty directory.


In [ ]:
if IN_COLAB:
    !pip3 install -r Projekt_AIM_1/requirements.txt
    !pip3 install optuna

  Using cached optuna-4.3.0-py3-none-any.whl.metadata (17 kB)
Using cached optuna-4.3.0-py3-none-any.whl (386 kB)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models
from torchvision.models import *
from plotly import express as px
from collections import Counter
import numpy as np
import random

if(IN_COLAB):
    from Projekt_AIM_1.modules.dataset import IntelImageClassificationDataset
    from Projekt_AIM_1.modules.utility import NotebookPlotter, InferenceSession, Evaluator, ISO_time, apply_pruning
    from Projekt_AIM_1.modules.trainer import Trainer
    from Projekt_AIM_1.modules.optuna_optimizer import OptunaTuner
    from Projekt_AIM_1.modules.my_optuna_tuner import run_optuna
else:
    from modules.dataset import IntelImageClassificationDataset
    from modules.utility import NotebookPlotter, InferenceSession, Evaluator, ISO_time, apply_pruning
    from modules.trainer import Trainer
    from modules.optuna_optimizer import OptunaTuner
    from modules.my_optuna_tuner import run_optuna

torch.manual_seed(1)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True  # for reproducibility
    torch.backends.cudnn.benchmark = False

# Removed support for Tensor Units
# torch.backends.cudnn.allow_tf32 = True
# torch.backends.cuda.matmul.allow_tf32 = True

set_seed(1)

ModuleNotFoundError: No module named 'mai_project1_optimization'

https://www.kaggle.com/datasets/puneet6060/intel-image-classification

In [ ]:
# labels, values = zip(*Counter([item[1] for item in dataset.train_dataset]).items())
# fig = px.bar(x=labels, y=values, labels={'x': 'Categories', 'y': 'Counts'}, title='Distribution of Classes')
# fig.show()

| n | label |
| --- | --- |
| 0 | buildings |
| 1 | forest |
| 2 | glacier |
| 3 | mountain |
| 4 | sea |
| 5 | street |

NotebookPlotter.plot_dataset_item_interactive(dataset.train_dataset)

In [ ]:
choice = 1 # 1,2,3
freezeLayer = False
prune_model = False
USE_OPTUNA = True

if choice != 5:
    dataset = IntelImageClassificationDataset(resize=(150,150))
else:
    dataset = IntelImageClassificationDataset(resize=(384,384))

# 80% train, 20% validation for training Optuna
train_size = int(0.8 * len(dataset.train_dataset))
val_size = len(dataset.train_dataset) - train_size
train_subset, val_subset = random_split(dataset.train_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(1))

def build_model():

  # SqueezeNet 1.1
  if choice == 1:
      model = models.squeezenet1_1(weights=SqueezeNet1_1_Weights.DEFAULT)
      num_features = model.classifier[1].in_channels
      kernel_size = model.classifier[1].kernel_size
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.classifier[1] = nn.Conv2d(num_features, 6, kernel_size)


  # MobileNetV2
  elif choice == 2:
      model = models.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)
      num_features = model.classifier[1].in_features
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.classifier[1] = nn.Linear(num_features, 6)

  # MobileNetV3 Small
  elif choice == 3:
      model = models.mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)
      num_features = model.classifier[3].in_features
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.classifier[3] = nn.Linear(num_features, 6)

  # MobileNetV3 Large
  elif choice == 4:
      model = models.mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.DEFAULT)
      num_features = model.classifier[3].in_features
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.classifier[3] = nn.Linear(num_features, 6)

  # VisionTransformer Base 16
  elif choice == 5:
      model = models.vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_SWAG_E2E_V1)
      num_features = model.heads[0].in_features
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.heads[0] = nn.Linear(num_features, 6)

  if prune_model:
    model = apply_pruning(model, amount=0.3)

  return model

if USE_OPTUNA:
    tuner = OptunaTuner(
        model_fn=build_model,
        train_dataset=train_subset,
        val_dataset=val_subset,
        device=DEVICE
    )
    study = tuner.run(n_trials=10, seed=1)
    print("Best trial parameters:", study.best_trial.params)

    best_params = study.best_trial.params
    model = build_model()
    dataloader = DataLoader(train_subset, batch_size=best_params["batch_size"], shuffle=True)
    trainer = Trainer(model=model, lr=best_params["lr"], device=DEVICE)
    epochs = best_params["epochs"]

elif OPTUNA_MO:
    model = build_model()
    best_params, best_model_state, study = run_optuna(
        model=model,
        dataset=dataset,
        TrainerClass=Trainer,
        n_trials=20
    )
    print("\u25b6 Optuna suggests:", best_params)

    model.load_state_dict(best_model_state)
    dataloader = DataLoader(dataset.train_dataset, batch_size=best_params["BS_SUGGEST"], shuffle=True)
    trainer = Trainer(model=model, lr=best_params["LR_SUGGEST"], device=DEVICE)
    epochs = 25

else:
    model = build_model()
    dataloader = DataLoader(dataset.train_dataset, batch_size=32, shuffle=True)
    trainer = Trainer(model=model, lr=8.841926348917726e-05, device=DEVICE)
    epochs = 25

[I 2025-05-04 10:34:14,971] A new study created in memory with name: no-name-06ee1d4b-5edc-48ab-bd92-eb7e142dc8a4
/content/Projekt_AIM_1/modules/optuna_optimizer.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_1-b8a52dc0.pth
100%|██████████| 4.73M/4.73M [00:00<00:00, 58.6MB/s]


  0%|          | 0/13 [00:00<?, ?it/s]

[I 2025-05-04 10:40:33,011] Trial 0 finished with value: 0.9102244389027432 and parameters: {'lr': 0.00017826497151386962, 'batch_size': 16, 'epochs': 13}. Best is trial 0 with value: 0.9102244389027432.


  0%|          | 0/21 [00:00<?, ?it/s]

[W 2025-05-04 10:44:14,308] Trial 1 failed with parameters: {'lr': 1.8924123965722247e-05, 'batch_size': 64, 'epochs': 21} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/content/Projekt_AIM_1/modules/optuna_optimizer.py", line 25, in objective
    trainer.train(train_loader, epochs=epochs, silent=True)
  File "/content/Projekt_AIM_1/modules/trainer.py", line 52, in train
    for _ in self.train_iter(dataloader, epochs=epochs, silent=silent):
  File "/content/Projekt_AIM_1/modules/trainer.py", line 87, in train_iter
    for i, (data, target) in enumerate(dataloader):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 708, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/

KeyboardInterrupt: 

In [ ]:
# model.load_state_dict(torch.load(f"checkpoints/.pt"))
trainer.train(dataloader, epochs=epochs, silent=False)

In [ ]:
session = InferenceSession(model)
output = session(torch.stack(tuple(item[0] for item in dataset.test_dataset)))
Evaluator.acc(output, torch.tensor(tuple(item[1] for item in dataset.test_dataset))).item()

0.9018919467926025

In [ ]:
# torch.save(model.state_dict(), f"checkpoints/{model.__class__.__name__}.pt")

## Initial Results for Model Selection

| model | accuracy | size |
| --- | --- | --- |
| ResNet18 | 0.87 | 44.7 MB |
| ResNet34 | 0.88 | 83.3 MB |
| MobileNet V2 | 0.91 | 13.6 MB |
| MobileNet V3 small | 0.90 | 9.8 MB |
| VGG19 | 0.83 | 548.1 MB |
| SqueezeNet 1.0 | 0.89 | 4.8 MB |
| DenseNet | 0.90 | 30.8 MB |
| EfficientNet B0 | 0.92 | 20.5 MB |
| ViT-b/16 | 0.73 | 330.3 MB |